In [49]:
import pandas as pd
import csv

In [50]:
def read_csv_from_sql(cols,path):
    df = pd.read_csv(path, header = None, quoting=csv.QUOTE_NONNUMERIC, encoding="utf-8", escapechar='\\', doublequote=False)
    df.columns = cols
    return df

In [51]:
df_list = []
name_list = []

In [52]:
columns = ["page_id", "page_namespace", "page_title", "page_restrictions", "page_counter", 
                "page_is_redirect", "page_is_new", "page_random", "page_touched", "page_links_updated", 
                "page_latest", "page_len", "page_content_model", "page_lang"]
patho = "../csv_from_sql/page_30k.csv"
df_page = read_csv_from_sql(columns,patho)
df_list.append(df_page)
name_list.append("page")

In [53]:
columns = ["pl_from", "pl_namespace", "pl_title", "pl_from_namespace"]
patho = "../csv_from_sql/pagelinks_30k.csv"
df_pagelinks = read_csv_from_sql(columns,patho)
df_list.append(df_pagelinks)
name_list.append("pagelinks")

In [54]:
columns = ["img_name", "img_size", "img_width", "img_height", "img_bits", "img_media_type", 
"img_major_mime", "img_minor_mime", "img_description", "img_user", "img_user_text", "img_timestamp", 
"img_sha1", "img_deleted"] 
patho = "../csv_from_sql/image_30k.csv"
df_image = read_csv_from_sql(columns,patho)
df_list.append(df_image)
name_list.append("image")

In [55]:
columns = ["il_from","il_to","il_from_namespace"]
patho = "../csv_from_sql/imagelinks_30k.csv"
df_imagelinks = read_csv_from_sql(columns,patho)
df_list.append(df_imagelinks)
name_list.append("imagelinks")

In [56]:
columns = ["cat_id", "cat_title", "cat_pages", "cat_subcats", "cat_files"]
patho = "../csv_from_sql/category_30k.csv"
df_category = read_csv_from_sql(columns,patho)
df_list.append(df_category)
name_list.append("category")

In [57]:
columns = ["cl_from","cl_to","cl_timestamp","cl_sortkey_prefix","cl_collation","cl_type"]
patho = "../csv_from_sql/categorylinks_30k.csv"
df_categorylinks = read_csv_from_sql(columns,patho)
df_list.append(df_categorylinks)
name_list.append("categorylinks")

In [58]:
from pymongo import MongoClient
import pymongo
client = MongoClient('localhost', 27017)

In [59]:
db = client['wiki_one_collection']


In [60]:
client.drop_database(u'wiki_one_collection')
client.database_names()
db = client['wiki_one_collection']
coll_name = "page"
coll = db[coll_name]
df = df_page
for x in range(0,len(df)):
    if x % 1000 == 0:
        print coll_name,"collection - progress:",float(x)/len(df)," (",x,"/",len(df),")"
    dictz = {}
    for y in df.columns:
        value = df[y][x]
        if isinstance(value, basestring) or isinstance(value, unicode):
            uffa = str(value.encode(encoding="utf-8"))
        else:
            uffa = float(value)
        dictz[y] = uffa
    list_of_embed_things = []
    dictz['categories'] = []
    list_of_cat = list(df_categorylinks[df_categorylinks["cl_from"]==df['page_id'][x]]["cl_to"])
    for c in list_of_cat:
        d_single_cat = {}
        df_single_cat = df_category[df_category[u'cat_title']==c]
        if len(df_single_cat) != 1:
            print "error! too many categories with same title:"
            print df_single_cat
            break
        for c_col in df_category.columns:
            value = list(df_single_cat[c_col])[0]
            if isinstance(value, basestring) or isinstance(value, unicode):
                uffa = str(value.encode(encoding="utf-8"))
            else:
                uffa = float(value)
            d_single_cat[c_col] = uffa
        dictz['categories'].append(d_single_cat)
    
    dictz['images'] = []
    list_of_imgs = list(df_imagelinks[df_imagelinks["il_from"]==df['page_id'][x]]["il_to"])
    for i in list_of_imgs:
        d_single_img = {}
        df_single_img = df_image[df_image[u'img_name']==i]
        if len(df_single_img) > 1:
            print "error! too many images with same title:"
            print df_single_img
            break
        elif len(df_single_img) == 0:
            continue
        else:
            for i_col in df_image.columns:
                value = list(df_single_img[i_col])[0]
                if isinstance(value, basestring) or isinstance(value, unicode):
                    uffa = str(value.encode(encoding="utf-8"))
                else:
                    uffa = float(value)
                d_single_img[i_col] = uffa
            dictz['images'].append(d_single_img)
            
    dictz['linked_pages'] = []
    list_of_pgs = list(df_pagelinks[df_pagelinks["pl_from"]==df['page_id'][x]]["pl_title"])
    for i in list_of_pgs:
        d_single_pg = {}
        df_single_pg = df_page[df_page[u'page_title']==i]
        if len(df_single_pg) > 1:
            print "error! too many pages with same title:"
            print df_single_pg
            break
        elif len(df_single_pg) == 0:
            continue
        else:
            for i_col in ["page_id","page_title"]:
                value = list(df_single_pg[i_col])[0]
                if isinstance(value, basestring) or isinstance(value, unicode):
                    uffa = str(value.encode(encoding="utf-8"))
                else:
                    uffa = float(value)
                d_single_pg[i_col] = uffa
            dictz['linked_pages'].append(d_single_pg)

    coll_id = coll.insert_one(dictz)


page collection - progress: 0.0  ( 0 / 30000 )
page collection - progress: 0.0333333333333  ( 1000 / 30000 )
page collection - progress: 0.0666666666667  ( 2000 / 30000 )
page collection - progress: 0.1  ( 3000 / 30000 )
page collection - progress: 0.133333333333  ( 4000 / 30000 )
page collection - progress: 0.166666666667  ( 5000 / 30000 )
page collection - progress: 0.2  ( 6000 / 30000 )
page collection - progress: 0.233333333333  ( 7000 / 30000 )
page collection - progress: 0.266666666667  ( 8000 / 30000 )
page collection - progress: 0.3  ( 9000 / 30000 )
page collection - progress: 0.333333333333  ( 10000 / 30000 )
page collection - progress: 0.366666666667  ( 11000 / 30000 )
page collection - progress: 0.4  ( 12000 / 30000 )
page collection - progress: 0.433333333333  ( 13000 / 30000 )
page collection - progress: 0.466666666667  ( 14000 / 30000 )
page collection - progress: 0.5  ( 15000 / 30000 )
page collection - progress: 0.533333333333  ( 16000 / 30000 )
page collection - progr

In [61]:
print "The collections in our mongoDB are:"
print
for x in db.collection_names():
    print x

The collections in our mongoDB are:

page


In [62]:
# command to create index directly in mongo shell
# db.page.createIndex( { 'page_id': 1 } )
print sorted(list(db["page"].index_information()))


[u'_id_']


In [63]:
#creating indexes in case it was not done already in mongo shell
collection_now = db["page"]
result = collection_now.create_index([('page_id', pymongo.ASCENDING)], unique=True)
print "indexes for collection of pages:" 
print sorted(list(collection_now.index_information()))
print

indexes for collection of pages:
[u'_id_', u'page_id_1']



In [64]:
i = 0
r = 0
for x in db.page.find(
                        {"$and":[{ "categories.cat_title": {"$regex" : ".*_War_.*"}},
                        {"page_title": {"$regex" : ".*_submarine_.*"}}]},
                        {"page_title":1,"_id":0}
                        ):
    print "result #",r
    r+=1
    print x
    print "----"
    i += 1
    if i >=100:
        print "break"
        break

result # 0
{u'page_title': u'Soviet_submarine_K-8'}
----
result # 1
{u'page_title': u'German_submarine_U-110_(1940)'}
----
result # 2
{u'page_title': u'German_submarine_U-573'}
----


In [79]:
'''

"categories.cat_title": "Redirects_from_misspellings"
"$where": "this.categories.length > 1"
"categories.cat_title": {"$regex" : ".*_War_.*"}

'''

'\n\n"categories.cat_title": "Redirects_from_misspellings"\n"$where": "this.categories.length > 1"\n\n'

In [ ]:
i = 0
# getting all pages related to war and showing the relative war categories
# https://docs.mongodb.com/manual/tutorial/query-documents/
# other query oprators here: https://docs.mongodb.com/manual/reference/operator/query/
r = 0
for x in db.page.find({ "categories.cat_title": {"$regex" : ".*_War_.*"} }):
    print "result #",r
    csteg = []
    for y in x["categories"]:
        if "_War_" in y["cat_title"]:
            csteg.append(y["cat_title"])
    print x["page_title"],":",csteg
    print "----"
    i += 1
    if i >=10:
        print "break"
        break


In [ ]:
i = 0
# full structured print
# https://docs.mongodb.com/manual/tutorial/query-documents/
# other query oprators here: https://docs.mongodb.com/manual/reference/operator/query/
r = 0
for x in db.page.find({ "categories.cat_title": {"$regex" : ".*_War_.*"} }):
    print "result #",r
    r+=1
    for y in x:
        if type(x[y]) != list:
            print y,"-->", x[y]
        else:
            if len(x[y]) == 0:
                print y,"--> EMPTY LIST"
            else:
                print
                print y,"-->"
                gn = 0
                for g in x[y]:
                    print " ", "item #",gn
                    gn+=1
                    for h in g:
                        print "  ",h,":",g[h]
                print
    print
    print "----"
    print
    i += 1
    if i >=10:
        print "break"
        break
    

In [45]:
dizio = {}
define( 'NS_MAIN', 0 );
define( 'NS_TALK', 1 );
define( 'NS_USER', 2 );
define( 'NS_USER_TALK', 3 );
define( 'NS_PROJECT', 4 );
define( 'NS_PROJECT_TALK', 5 );
define( 'NS_FILE', 6 );
define( 'NS_FILE_TALK', 7 );
define( 'NS_MEDIAWIKI', 8 );
define( 'NS_MEDIAWIKI_TALK', 9 );
define( 'NS_TEMPLATE', 10 );
define( 'NS_TEMPLATE_TALK', 11 );
define( 'NS_HELP', 12 );
define( 'NS_HELP_TALK', 13 );
define( 'NS_CATEGORY', 14 );
define( 'NS_CATEGORY_TALK', 15 );

In [ ]:
def define(stringaz,numbez):
    lol = ""
    for y in stringaz.split("_")[1:]:
        lol+=" "+y
    dizio[numbez] = lol

In [46]:
c = 0
for x in db.page.aggregate(pipeline):
    try:
        print dizio[x["_id"]],":",x['count']
        
    except KeyError:
        print " "+str(x["_id"]),":",x['count']
    c+= x['count']
print "--------------"
print "=",c

 MAIN : 15309
 TALK : 2781
 CATEGORY : 723
 FILE : 501
 USER TALK : 327
 USER : 125
 PROJECT : 117
 TEMPLATE : 116
 CATEGORY TALK : 38
 PROJECT TALK : 23
 TEMPLATE TALK : 17
 100.0 : 13
 101.0 : 4
 HELP : 2
 HELP TALK : 1
 MEDIAWIKI TALK : 1
 MEDIAWIKI : 1
 FILE TALK : 1
--------------
= 20100
